В данной работе описана следующая схема: сначала на выбранном датасете обучается ансамбль глубоких сверточных сетей, после чего неглубокая полносвязная сеть (с одним скрытым слоем) обучается выдавать те же ответы, что и ансамбль глубоких сетей. Точность обученной таким образом неглубокой сети хоть и отстает от точности ансамбля глубоких сетей, но опережает точность такой же сети, обученной на исходном датасете.

Для экспериментов использовались два датасета:
- Датасет TIMIT: распознавание речи
- Датасет CIFAR-10: классификация изображений

Рассмотрим архитектуру, использовавшуюся на первом датасете. На вход подавался вектор длиной 1845.

*Глубокая сверточная сеть* состояла из сверточных слоев (за деталями ее архитектуры авторы отсылают к работе [Recent advances in deep learning for speech research at microsoft]($Recent advances in deep learning for speech research at microsoft$)), трех скрытых полносвязных слоев с функцией активации ReLU и выходного слоя для задачи классификации со 183 классами. На датасете обучались несколько таких глубоких сетей (не уточняется были ли в их архитектурах какие-то отличия).

*Неглубокая сеть* состояла из одного скрытого слоя размером 400 000 нейронов с функцией активации ReLU и выходного слоя. Матрица весов скрытого слоя была факторизована в виде произведения двух матриц размером 1845x250 и 250x400000. Таким образом достигалось ускорение обучения, уменьшение времени вычислений и занимаемой памяти. Неглубокая сеть обучалась минимизацией L2-loss (среднеквадратичного отклонения) между logit'ами неглубокой сети и ансамбля глубоких сетей. Для обучения использовались те же данные.

<img src="assets/deepnets.jpg" width="500" align="center">

При обучении CIFAR-10 все было устроено в целом аналогично, но на стадии обучения неглубокой сети вместо размеченных изображений использовались неразмеченные, которых доступно на порядки больше.

Особенно интересным кажется результат, достигнутый на TIMIT, поскольку для обучения неглубокой сети в нем не использовалось никаких дополнительных данных. Получается, что если мы хотим обучить неглубокую сеть на таком датасете как TIMIT, то обучать ее стандартным способом (минимизацией кроссэнтропии) - не лучший вариант. Более высокая точность достигается если сначала обучить глубокую сеть, а потом обучить неглубокую сеть "копировать" ответ глубокой, используя те же входные данные.

### Почему это работает?

Если бы мы минимизировали L2-отклонение не на logits, а на вероятностях (т. е. после softmax), то все сводилось бы к тому, что оригинальную разметку датасета мы заменяем разметкой, полученной из глубокой модели. В случае минимизации L2-отклонения на logits ситуация, по-видимому, не меняется принципиально. С первого взгляда кажется, что замена оригинальной разметки на взятую из модели разметку должно только ухудшить качество обучения, так как в разметку добавляются ошибки, сделанные моделью. Почему же это работает?

Авторы перечисляют возможные объяснения:

- Глубокая сеть может, наоборот, исправлять ошибки, допущенные в разметке датасета. Известно, что в датасете TIMIT есть неверно размеченные примеры.
- Может играть роль тот факт, что глубокая сеть ("модель-учитель") дает менее уверенные ответы, распределяя вероятности по разным классам. При этом увеличивается количество полезной информации, доступной неглубокой сети ("модели-студенту"), так как появляется информация о распределении вероятностей по оставшимся (неверным) классам. Пример из датасета может быть похож сразу на несколько классов, например на класс 1 с вероятностью 60% и на класс 2 с вероятностью 40%. В строгой разметке (1 или 0) это не будет отражено, а в logit'ах глубокой сети эта информация будет присутствовать.

Вторая гипотеза перекликается с техникой регуляризации label smoothing, предложенной позднее в статье [Rethinking the Inception Architecture for Computer Vision]($Rethinking the Inception Architecture for Computer Vision$). Добавлю к этому списку гипотез также и свое предположение о том, что logits содержат больше информации, чем вероятности, так как прибавление ко всем logits одного и тот же (любого) числа не повлияет на вероятности, следовательно при переходе от logits к вероятностям часть информации теряется.

Авторы делают вывод о том, что описанный способ обучения неглубокой сети может являться способом регуляризации, по сравнению с ее обучением на исходном датасете.

Если глубокую сеть авторы обучали 8-12 часов, то неглубокая сеть учится копировать глубокую существенно быстрее (1-2 часа). Также в неглубокой сети быстрее осуществляется инференс. В заключение авторы говорят о том, что большим успехом было бы нахождение алгоритма обучения, который способен обучить неглубокую сеть до такой же точности напрямую на исходных данных, без привлечения модели-учителя.

Также интересен следующий вопрос: если в качестве модели-ученика взять еще одну глубокую модель, то может ли она превзойти по точности модель-учителя?